MUlti layer inference

In [ ]:
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 4
    param_size_GB = param_size_bytes / (1024 ** 3)
    return int(device_memory/param_size_GB)-drop

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])

    def forward(self, x, cos, sin,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin))[0]
        return x
layers=model.model.layers


# Usage
# compiled_block = torch.compile(FusedBlock(layers[i:j]))

def compute(model, x, layers_per_chunk, cos, sin,k,max_new_tokens,attention_mask):
    total_layers = len(model.model.layers)
    if layers_per_chunk >= total_layers:
        # model = model.to('cuda')
        # out = model({'inputs_embeds': x})['logits'][:, -1, :]
        # return out.argmax(-1)
        i=0
        l=total_layers
        m=Module(model.model.layers,i,i+l).to('cuda')
        x=m(x.to('cuda'),cos.to('cuda'),sin.to('cuda'),attention_mask.to('cuda'))
        if(k==max_new_tokens-1):
            m=m.to('cpu')
            torch.cuda.empty_cache()
        return x
    i = 0
    rem = total_layers
    while rem > 0:
        l = min(layers_per_chunk, rem)
        m = Module(model.model.layers, i, i + l).to('cuda')
        # m=torch.compile(m)
        # Run chunk
        x = m(x.to('cuda'), cos.to('cuda'), sin.to('cuda'),attention_mask)
        # if(k==max_new_tokens-1):
        #     m=m.to('cpu')
        #     torch.cuda.empty_cache()
        m=m.to('cpu')
        torch.cuda.empty_cache()
        # update indices
        i += l
        rem -= l

    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu')
    input_ids=prompt['input_ids'].to('cuda')
    attention_mask=prompt['attention_mask']
    del prompt
    for i in range(tokens):
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        # input_ids.to('cpu')
        # torch.cuda.empty_cache()
        # model.model.embed_tokens.to('cpu')
        # torch.cuda.empty_cache()
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        rotary_emb = model.model.rotary_emb
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        cnt=0
        with torch.no_grad():
            l1=get_layers(model)
            x=compute(model,x,15,cos,sin,i,tokens,attention_mask)
            model.model.norm.to('cuda')
            x=model.model.norm(x)
            # model.model.norm.to('cpu')
            # torch.cuda.empty_cache()
            model.lm_head.to('cuda')
            x=model.lm_head(x)
            # model.lm_head.to('cpu')
            # torch.cuda.empty_cache()
        
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1) 
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
        

Single layer infernece

In [ ]:
def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu')
    input_ids=prompt['input_ids'].to('cuda')
    attention_mask=prompt['attention_mask']
    del prompt
    for i in range(tokens):
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cuda')
        x = model.model.embed_tokens(input_ids)
        input_ids.to('cpu')
        # del input_ids
        torch.cuda.empty_cache()
        model.model.embed_tokens.to('cpu')
        torch.cuda.empty_cache()
        # Safe check
        seq_len = x.shape[1]
        batch_size = x.shape[0]
        position_ids = torch.arange(seq_len, dtype=torch.long, device=x.device).unsqueeze(0).expand(batch_size, -1)
        rotary_emb = model.model.rotary_emb
        cos, sin = rotary_emb(x=x,position_ids=position_ids)
        cnt=0
        for i in model.model.layers:
            cnt+=1
            # print(cnt)
            i.to('cuda')
            x.to('cuda')
            # print(x)
            with torch.no_grad():
                x=i(x,position_embeddings=(cos,sin))
                # print(x)
            i.to('cpu')
            torch.cuda.empty_cache()
            x=x[0]
            # x.to('cpu')
            torch.cuda.empty_cache()
            
        with torch.no_grad():
            model.model.norm.to('cuda')
            x=model.model.norm(x)
            model.model.norm.to('cpu')
            torch.cuda.empty_cache()
            model.lm_head.to('cuda')
            x=model.lm_head(x)
            model.lm_head.to('cpu')
            torch.cuda.empty_cache()
        
        x1=x[:,-1,:]
        x1=torch.argmax(x1,dim=-1,keepdim=True)
        print(tok.decode(x1[0]),end="")
        input_ids=torch.cat([input_ids,x1],dim=-1) 
        # x1=x.argmax(-1)
        # print(tok.decode(x1[0]),end="\n")
        # question=question+" "+tok.decode(x1[0])
        

In [ ]:
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory/param_size_GB)-drop)//2

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
       
    # @torch.compile
    def forward(self, x, cos, sin,dynamic_cache,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin),past_key_value=dynamic_cache,use_cache=True,)[0]
        return x
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
layers=model.model.layers
def slice_past(past_list, start, end):
    return past_list[start:end] if past_list else None

def compute(model_instance, x, layers_per_chunk, cos, sin, k, dynamic_cache,max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda', non_blocking=True)
        x = m(x, cos, sin,dynamic_cache,attention_mask)
        if k == max_new_tokens - 1:
            m = m.to('cpu', non_blocking=True)
        return x

    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_next2 = None
    m_previous = None

    # Load first two chunks
    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first
        
        if rem > 0:
            l_second = min(layers_per_chunk, rem)
            # wait to avoid overlapping transfers
            stream_transfer2.wait_stream(stream_transfer)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_second).to('cuda', non_blocking=True)
            i += l_second
            rem -= l_second

    while True:
        stream_compute.wait_stream(stream_transfer)
        stream_compute.wait_stream(stream_transfer2)

        m_previous = m_current
        m_current = m_next
        m_next = m_next2
        m_next2 = None

        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(
                    x,
                    cos,
                    sin,
                    dynamic_cache,
                    attention_mask
                )
        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                del m_previous
        torch.cuda.current_stream().synchronize()
        torch.cuda.empty_cache()
        if rem == 0 and m_next is None and m_next2 is None:
            stream_compute.synchronize()
            with torch.cuda.stream(stream_offload):
                m_current = m_current.to('cpu', non_blocking=True)
                del m_current
            stream_offload.synchronize()
            torch.cuda.empty_cache()
            break
        if rem > 0:
            l_next2 = min(layers_per_chunk, rem)
            stream_transfer2.wait_stream(stream_transfer)
            with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(model_instance.model.layers, i, i + l_next2).to('cuda', non_blocking=True)
            i += l_next2
            rem -= l_next2
    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu',non_blocking=True)
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda',non_blocking=True)
    model.model.norm.to('cuda',non_blocking=True)
    model.lm_head.to('cuda',non_blocking=True)
    rotary_emb = model.model.rotary_emb.to('cuda',non_blocking=True)
    dynamic_cache = DynamicCache()
    current_input_ids = input_ids
    for k in range(tokens):
        torch.cuda.empty_cache()
        if k==0:
            seq_len = current_input_ids.shape[1]
            batch_size = current_input_ids.shape[0]
            current_position_id=torch.arange(seq_len, dtype=torch.long, device=current_input_ids.device).unsqueeze(0).expand(batch_size, -1)
        else:
            current_position_id = torch.tensor([[dynamic_cache.get_seq_length()]], device='cuda')
        x = model.model.embed_tokens(current_input_ids)
        cos, sin = rotary_emb(x=x, position_ids=current_position_id)
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,3,cos,sin,k,dynamic_cache,tokens,attention_mask)
            x=model.model.norm(x)
            logits=model.lm_head(x)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        print(tokenizer.decode(next_token_id[0]), end="")
        current_input_ids = next_token_id
    model.model.norm.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()

In [ ]:
# 5 workers

def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory/param_size_GB)-drop)//2

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
       
    # @torch.compile
    def forward(self, x, cos, sin,dynamic_cache,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin),past_key_value=dynamic_cache,use_cache=True,)[0]
        return x
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
layers=model.model.layers
def slice_past(past_list, start, end):
    return past_list[start:end] if past_list else None

def compute(model_instance, x, layers_per_chunk, cos, sin, k, dynamic_cache,max_new_tokens, attention_mask):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    stream_transfer3 = torch.cuda.Stream()  # New 3rd stream
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda', non_blocking=True)
        x = m(x, cos, sin,dynamic_cache,attention_mask)
        if k == max_new_tokens - 1:
            m = m.to('cpu', non_blocking=True)
        return x

    i = 0
    rem = total_layers
    m_current = None
    m_next = None
    m_next2 = None
    m_next3 = None  # New 3rd module
    m_previous = None
    # Load first three chunks
    if rem > 0:
        l_first = min(layers_per_chunk, rem)
        with torch.cuda.stream(stream_transfer):
            m_next = Module(model_instance.model.layers, i, i + l_first).to('cuda', non_blocking=True)
        i += l_first
        rem -= l_first
        
    if rem > 0:
        l_second = min(layers_per_chunk, rem)
        stream_transfer2.wait_stream(stream_transfer)
        with torch.cuda.stream(stream_transfer2):
            m_next2 = Module(model_instance.model.layers, i, i + l_second).to('cuda', non_blocking=True)
        i += l_second
        rem -= l_second
            
    if rem > 0:
        l_third = min(layers_per_chunk, rem)  # Load only 1 layer
        stream_transfer3.wait_stream(stream_transfer2)
        with torch.cuda.stream(stream_transfer3):
            m_next3 = Module(model_instance.model.layers, i, i + l_third).to('cuda', non_blocking=True)
        i += l_third
        rem -= l_third

    while True:
        # stream_compute.wait_stream(stream_transfer)
        stream_compute.wait_stream(stream_transfer2)
        # stream_compute.wait_stream(stream_transfer3)  # Wait for 3rd stream too

        m_previous = m_current
        m_current = m_next
        m_next = m_next2
        m_next2 = m_next3
        m_next3 = None

        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(
                    x,
                    cos,
                    sin,
                    dynamic_cache,
                    attention_mask
                )
                
        if m_previous is not None:
            stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                del m_previous
                
        # torch.cuda.current_stream().synchronize()
        torch.cuda.empty_cache()
        
        if rem == 0 and m_next is None and m_next2 is None and m_next3 is None:
            # stream_compute.synchronize()
            # with torch.cuda.stream(stream_offload):
            #     m_current = m_current.to('cpu', non_blocking=True)
            #     del m_current
            # stream_offload.synchronize()
            # torch.cuda.empty_cache()
            break
            
        # Load next chunk on the 3rd stream (1 layer only)
        if rem > 0:
            l_next3 = min(layers_per_chunk, rem)  # Always 1 layer
            stream_transfer3.wait_stream(stream_transfer)
            with torch.cuda.stream(stream_transfer3):
                m_next3 = Module(model_instance.model.layers, i, i + l_next3).to('cuda', non_blocking=True)
            i += l_next3
            rem -= l_next3
            
    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu',non_blocking=True)
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda',non_blocking=True)
    model.model.norm.to('cuda',non_blocking=True)
    model.lm_head.to('cuda',non_blocking=True)
    rotary_emb = model.model.rotary_emb.to('cuda',non_blocking=True)
    dynamic_cache = DynamicCache()
    current_input_ids = input_ids
    for k in range(tokens):
        # torch.cuda.empty_cache()
        if k==0:
            seq_len = current_input_ids.shape[1]
            batch_size = current_input_ids.shape[0]
            current_position_id=torch.arange(seq_len, dtype=torch.long, device=current_input_ids.device).unsqueeze(0).expand(batch_size, -1)
        else:
            current_position_id = torch.tensor([[dynamic_cache.get_seq_length()]], device='cuda')
        x = model.model.embed_tokens(current_input_ids)
        cos, sin = rotary_emb(x=x, position_ids=current_position_id)
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,2,cos,sin,k,dynamic_cache,tokens,attention_mask)
            x=model.model.norm(x)
            logits=model.lm_head(x)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        print(tokenizer.decode(next_token_id[0]), end="")
        current_input_ids = next_token_id
    model.model.embed_tokens.to('cpu',non_blocking=True)
    model.model.norm.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    rotary_emb.to('cpu',non_blocking=True)

preloading

In [1]:
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory/param_size_GB)-drop)//2

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
       
    # @torch.compile
    def forward(self, x, cos, sin,dynamic_cache,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin),past_key_value=dynamic_cache,use_cache=True,)[0]
        return x
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
layers=model.model.layers
def slice_past(past_list, start, end):
    return past_list[start:end] if past_list else None



def compute(model_instance, x, layers_per_chunk, cos, sin, k, dynamic_cache,max_new_tokens, attention_mask,modules):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    stream_transfer3 = torch.cuda.Stream()  # New 3rd stream
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda', non_blocking=True)
        x = m(x, cos, sin,dynamic_cache,attention_mask)
        if k == max_new_tokens - 1:
            m = m.to('cpu', non_blocking=True)
        return x

    # i = 0
    
    m_current = None
    m_next = None
    m_next2 = None
    m_next3 = None
    m_previous = None
    rem = len(modules)
    i=0
    if rem > 0:
        with torch.cuda.stream(stream_transfer):
            m_next = modules[i].to('cuda', non_blocking=True)
        i+=1
        rem -= 1
        
    if rem > 0:
        # stream_transfer2.wait_stream(stream_transfer)
        with torch.cuda.stream(stream_transfer2):
            m_next2 = modules[i].to('cuda', non_blocking=True)
        i += 1
        rem -= 1
                
    # if rem > 0:
    #     # stream_transfer3.wait_stream(stream_transfer2)
    #     with torch.cuda.stream(stream_transfer3):
    #         m_next3 = modules[i].to('cuda', non_blocking=True)
    #     i += 1
    #     rem -= 1

    while True:
        # stream_compute.wait_stream(stream_transfer)
        stream_compute.wait_stream(stream_transfer2)
        # stream_compute.wait_stream(stream_transfer3)  # Wait for 3rd stream too

        m_previous = m_current
        m_current = m_next
        m_next = m_next2
        m_next2 = None
        # m_next3 = None

        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                x = m_current(
                    x,
                    cos,
                    sin,
                    dynamic_cache,
                    attention_mask
                )
                
        if m_previous is not None:
            # stream_offload.wait_stream(stream_compute)
            with torch.cuda.stream(stream_offload):
                m_previous = m_previous.to('cpu', non_blocking=True)
                # del m_previous
        torch.cuda.empty_cache()
        
        if rem == 0 and m_next is None and m_next2 is None and m_next3 is None:
            if k==max_new_tokens-1:
                stream_compute.synchronize()
                with torch.cuda.stream(stream_offload):
                    m_current = m_current.to('cpu', non_blocking=True)
                    del m_current
                stream_offload.synchronize()
                torch.cuda.empty_cache()
                break
            break
        if rem > 0 :
            # stream_transfer3.wait_stream(stream_compute)
            with torch.cuda.stream(stream_transfer):
                m_next2 = modules[i].to('cuda', non_blocking=True)
            i += 1
            rem -= 1
            
    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu',non_blocking=True)
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda',non_blocking=True)
    model.model.norm.to('cuda',non_blocking=True)
    model.lm_head.to('cuda',non_blocking=True)
    rotary_emb = model.model.rotary_emb.to('cuda',non_blocking=True)
    dynamic_cache = DynamicCache()
    current_input_ids = input_ids
    modules=[]
    rem1=len(model.model.layers)
    i=0
    while(rem1>=1):
        l=min(2,rem1)
        modules.append(Module(model.model.layers,i,i+l).to('cpu',non_blocking=True))
        i+=l
        rem1-=l
    for k in range(tokens):
        if k==0:
            seq_len = current_input_ids.shape[1]
            batch_size = current_input_ids.shape[0]
            current_position_id=torch.arange(seq_len, dtype=torch.long, device=current_input_ids.device).unsqueeze(0).expand(batch_size, -1)
        else:
            current_position_id = torch.tensor([[dynamic_cache.get_seq_length()]], device='cuda')
        x = model.model.embed_tokens(current_input_ids)
        cos, sin = rotary_emb(x=x, position_ids=current_position_id)
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,1,cos,sin,k,dynamic_cache,tokens,attention_mask,modules)
            x=model.model.norm(x)
            logits=model.lm_head(x)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        print(tokenizer.decode(next_token_id[0]), end="")
        current_input_ids = next_token_id
    model.model.embed_tokens.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.model.norm.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    rotary_emb.to('cpu',non_blocking=True)

NameError: name 'nn' is not defined

In [ ]:
def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory/param_size_GB)-drop)//2

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
       
    # @torch.compile
    def forward(self, x, cos, sin,dynamic_cache,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin),past_key_value=dynamic_cache,use_cache=True,)[0]
        return x
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
layers=model.model.layers

def compute(model_instance, x, layers_per_chunk, cos, sin, k, dynamic_cache,max_new_tokens, attention_mask,modules):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    # stream_transfer3 = torch.cuda.Stream()  # New 3rd stream
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda', non_blocking=True)
        x = m(x, cos, sin,dynamic_cache,attention_mask)
        if k == max_new_tokens - 1:
            m = m.to('cpu', non_blocking=True)
        return x

    # i = 0
    
    m_current = None
    m_current2=None
    m_next = None
    m_next2 = None
    m_previous = None
    m_previous2=None
    rem = len(modules)
    i=0
    if rem > 0:
        with torch.cuda.stream(stream_transfer):
            m_next = modules[i].to('cuda', non_blocking=True)
            m_next2 = modules[i+1].to('cuda', non_blocking=True)# [0,1,2,3,4,5,6,7]
        i+=2
        rem -= 2
    while True:
        stream_compute.wait_stream(stream_transfer)

        m_previous = m_current
        m_previous2=m_current2
        m_current = m_next
        m_current2 = m_next2
        m_next=None
        m_next2 = None
        
        
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                if(m_current!=None):
                    x = m_current(
                        x,
                        cos,
                        sin,
                        dynamic_cache,
                        attention_mask
                    )
                if(m_current2!=None):
                    x=m_current2(
                        x,
                        cos,
                        sin,
                        dynamic_cache,
                        attention_mask
                    )
        if m_previous is not None:
            with torch.cuda.stream(stream_offload):
                m_previous.to('cpu',non_blocking=True)
        if m_previous2 is not None:
            with torch.cuda.stream(stream_offload):
                m_previous2.to('cpu',non_blocking=True)
        torch.cuda.empty_cache()
        if rem == 0 and m_next is None and m_next2 is None :
            if k==max_new_tokens-1:
                stream_compute.synchronize()
                with torch.cuda.stream(stream_offload):
                    m_current = m_current.to('cpu', non_blocking=True)
                stream_offload.synchronize()
                torch.cuda.empty_cache()
                break
            break
        if rem > 0:
            # stream_transfer.wait_stream(stream_offload)
            with torch.cuda.stream(stream_transfer):
                m_next = modules[i].to('cuda', non_blocking=True)
                m_next2 = modules[i+1].to('cuda', non_blocking=True)# [0,1,2,3,4,5,6,7] 
            i+=2
            rem -= 2
    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu',non_blocking=True)
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda',non_blocking=True)
    model.model.norm.to('cuda',non_blocking=True)
    model.lm_head.to('cuda',non_blocking=True)
    rotary_emb = model.model.rotary_emb.to('cuda',non_blocking=True)
    dynamic_cache = DynamicCache()
    current_input_ids = input_ids
    modules=[]
    rem1=len(model.model.layers)
    i=0
    while(rem1>=1):
        l=min(2,rem1)
        modules.append(Module(model.model.layers,i,i+l).to('cpu',non_blocking=True))
        i+=l
        rem1-=l
    for k in range(tokens):
        if k==0:
            seq_len = current_input_ids.shape[1]
            batch_size = current_input_ids.shape[0]
            current_position_id=torch.arange(seq_len, dtype=torch.long, device=current_input_ids.device).unsqueeze(0).expand(batch_size, -1)
        else:
            current_position_id = torch.tensor([[dynamic_cache.get_seq_length()]], device='cuda')
        x = model.model.embed_tokens(current_input_ids)
        cos, sin = rotary_emb(x=x, position_ids=current_position_id)
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,21,cos,sin,k,dynamic_cache,tokens,attention_mask,modules)
            x=model.model.norm(x)
            logits=model.lm_head(x)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        print(tokenizer.decode(next_token_id[0]), end="")
        current_input_ids = next_token_id
    model.model.embed_tokens.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.model.norm.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    rotary_emb.to('cpu',non_blocking=True)

In [ ]:
#gpu efficient

def get_layers(model,drop=2):
    device_memory=(torch.cuda.get_device_properties('cuda').total_memory)/(1024**3)
    l=model.model.layers[0]
    num_params = sum(p.numel() for p in l.parameters())
    param_size_bytes = num_params * 2
    param_size_GB = param_size_bytes / (1024 ** 3)
    return (int(device_memory/param_size_GB)-drop)//2

class Module(nn.Module):
    def __init__(self, layers, i, j):
        super().__init__()
        self.model = nn.ModuleList(layers[i:j])
       
    # @torch.compile
    def forward(self, x, cos, sin,dynamic_cache,attention_mask):
        for layer in self.model:
            x = layer(x, position_embeddings=(cos, sin),past_key_value=dynamic_cache,use_cache=True,)[0]
        return x
tokenizer=AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
layers=model.model.layers

def compute(model_instance, x, layers_per_chunk, cos, sin, k, dynamic_cache,max_new_tokens, attention_mask,modules):
    total_layers = len(model_instance.model.layers)
    stream_compute = torch.cuda.Stream()
    stream_transfer = torch.cuda.Stream()
    stream_transfer2 = torch.cuda.Stream()
    # stream_transfer3 = torch.cuda.Stream()  # New 3rd stream
    stream_offload = torch.cuda.Stream()
    
    if layers_per_chunk >= total_layers:
        i = 0
        l = total_layers
        m = Module(model_instance.model.layers, i, i + l).to('cuda', non_blocking=True)
        x = m(x, cos, sin,dynamic_cache,attention_mask)
        if k == max_new_tokens - 1:
            m = m.to('cpu', non_blocking=True)
        return x

    # i = 0
    
    m_current = None
    m_current2=None
    m_next = None
    m_next2 = None
    m_previous = None
    m_previous2=None
    rem = len(modules)
    i=0
    if rem > 0:
        with torch.cuda.stream(stream_transfer):
            full_mod = modules[i]  # This is already your Module wrapper
            total_layers_in_mod = len(full_mod.model)
            half = total_layers_in_mod // 2

            m_next = Module(full_mod.model, 0, half).to('cuda', non_blocking=True)
        # with torch.cuda.stream(stream_transfer2):
            m_next2 = Module(full_mod.model, half, total_layers_in_mod).to('cuda', non_blocking=True)
        i += 1
        rem -= 1
    while True:
        stream_compute.wait_stream(stream_transfer)

        m_previous = m_current
        m_previous2=m_current2
        m_current = m_next
        m_current2 = m_next2
        m_next=None
        m_next2 = None
        
        
        with torch.cuda.stream(stream_compute):
            with torch.no_grad():
                if(m_current!=None):
                    x = m_current(
                        x,
                        cos,
                        sin,
                        dynamic_cache,
                        attention_mask
                    )
                if(m_current2!=None):
                    x=m_current2(
                        x,
                        cos,
                        sin,
                        dynamic_cache,
                        attention_mask
                    )
        if m_previous is not None:
            with torch.cuda.stream(stream_offload):
                m_previous.to('cpu',non_blocking=True)
        if m_previous2 is not None:
            with torch.cuda.stream(stream_offload):
                m_previous2.to('cpu',non_blocking=True)
        torch.cuda.empty_cache()
        if rem == 0 and m_next is None and m_next2 is None :
            if k==max_new_tokens-1:
                stream_compute.synchronize()
                with torch.cuda.stream(stream_offload):
                    m_current = m_current.to('cpu', non_blocking=True)
                stream_offload.synchronize()
                torch.cuda.empty_cache()
                break
            break
        if rem > 0:
            with torch.cuda.stream(stream_transfer):
                full_mod = modules[i]  # This is already your Module wrapper
                total_layers_in_mod = len(full_mod.model)
                half = total_layers_in_mod // 2

                m_next = Module(full_mod.model, 0, half).to('cuda', non_blocking=True)
            # with torch.cuda.stream(stream_transfer2):
                m_next2 = Module(full_mod.model, half, total_layers_in_mod).to('cuda', non_blocking=True)
            i += 1
            rem -= 1
    return x

def model_inference(question,tokens=250):
    prompt=tok(question,return_tensors='pt').to('cpu',non_blocking=True)
    input_ids=prompt['input_ids'].to('cuda',non_blocking=True)
    attention_mask=prompt['attention_mask']
    model.model.embed_tokens.to('cuda',non_blocking=True)
    model.model.norm.to('cuda',non_blocking=True)
    model.lm_head.to('cuda',non_blocking=True)
    rotary_emb = model.model.rotary_emb.to('cuda',non_blocking=True)
    dynamic_cache = DynamicCache()
    current_input_ids = input_ids
    modules=[]
    rem1=len(model.model.layers)
    i=0
    while(rem1>=1):
        l=min(2,rem1)
        modules.append(Module(model.model.layers,i,i+l).to('cpu',non_blocking=True))
        i+=l
        rem1-=l
    for k in range(tokens):
        if k==0:
            seq_len = current_input_ids.shape[1]
            batch_size = current_input_ids.shape[0]
            current_position_id=torch.arange(seq_len, dtype=torch.long, device=current_input_ids.device).unsqueeze(0).expand(batch_size, -1)
        else:
            current_position_id = torch.tensor([[dynamic_cache.get_seq_length()]], device='cuda')
        x = model.model.embed_tokens(current_input_ids)
        cos, sin = rotary_emb(x=x, position_ids=current_position_id)
        with torch.no_grad():
            l1=get_layers(model,7)
            x=compute(model,x,1,cos,sin,k,dynamic_cache,tokens,attention_mask,modules)
            x=model.model.norm(x)
            logits=model.lm_head(x)
        next_token_id = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
        print(tokenizer.decode(next_token_id[0]), end="")
        current_input_ids = next_token_id
    model.model.embed_tokens.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.model.norm.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    model.lm_head.to('cpu',non_blocking=True)
    torch.cuda.empty_cache()
    rotary_emb.to('cpu',non_blocking=True)